In [ ]:
import pandas as pd
import requests
import time
import boto3
from bs4 import BeautifulSoup
import json
import re
import psycopg2

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Gathering cities's lattitude and longitude

In [ ]:
city_data = []
nominatim_url = "https://nominatim.openstreetmap.org/search"

def fetch_coordinates(city, retries=3):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    for attempt in range(retries):
        try:
            params = {
                "q": city,
                "format": "json",
                "countrycodes": "FR",
                "limit": 1
            }
            response = requests.get(nominatim_url, headers=headers, params=params)
            response.raise_for_status()  
            response_data = response.json()
            
            if response_data:
                lat = response_data[0].get("lat")
                lon = response_data[0].get("lon")
                return lat, lon
            
            
            print(f"No data found for {city}")
            time.sleep(1)
        
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt + 1} failed for {city}: {e}")
            time.sleep(2)  
    return None, None  


for idx, city in enumerate(cities, start=1):
    print(f"Fetching coordinates for {city} (City ID: {idx})...")
    lat, lon = fetch_coordinates(city)
    if lat and lon:
        city_data.append({"city_id": idx, "city_name": city, "city_latitude": lat, "city_longitude": lon})
    else:
        print(f"Failed to fetch data for {city} after multiple attempts")
    
    
    if idx % 10 == 0:
        print("Pausing to avoid rate limiting...")
        time.sleep(10)


city_df = pd.DataFrame(city_data)
print("Data fetching complete.")
city_df

Fetching coordinates for Mont Saint Michel (City ID: 1)...
Fetching coordinates for St Malo (City ID: 2)...
Fetching coordinates for Bayeux (City ID: 3)...
Fetching coordinates for Le Havre (City ID: 4)...
Fetching coordinates for Rouen (City ID: 5)...
Fetching coordinates for Paris (City ID: 6)...
Fetching coordinates for Amiens (City ID: 7)...
Fetching coordinates for Lille (City ID: 8)...
Fetching coordinates for Strasbourg (City ID: 9)...
Fetching coordinates for Chateau du Haut Koenigsbourg (City ID: 10)...
Pausing to avoid rate limiting...
Fetching coordinates for Colmar (City ID: 11)...
Fetching coordinates for Eguisheim (City ID: 12)...
Fetching coordinates for Besancon (City ID: 13)...
Fetching coordinates for Dijon (City ID: 14)...
Fetching coordinates for Annecy (City ID: 15)...
Fetching coordinates for Grenoble (City ID: 16)...
Fetching coordinates for Lyon (City ID: 17)...
Fetching coordinates for Gorges du Verdon (City ID: 18)...
Fetching coordinates for Bormes les Mimosa

,city_id,city_name,city_latitude,city_longitude
0,1,Mont Saint Michel,48.6359541,-1.511459954959514
1,2,St Malo,48.649518,-2.0260409
2,3,Bayeux,49.2764624,-0.7024738
3,4,Le Havre,49.4938975,0.1079732
4,5,Rouen,49.4404591,1.0939658
5,6,Paris,48.8534951,2.3483915
6,7,Amiens,49.8941708,2.2956951
7,8,Lille,50.6365654,3.0635282
8,9,Strasbourg,48.584614,7.7507127
9,10,Chateau du Haut Koenigsbourg,48.249410749999996,7.344320233724503


In [17]:
city_df.to_csv('city_information.csv', index=False)

### Gathering of booking.com hotels web page for each city

In [24]:
def test_geckodriver_direct():
    try:
        
        driver = webdriver.Firefox(service=Service("/snap/bin/geckodriver"))
        driver.get("https://www.google.com")
        
        
        time.sleep(5)
        
        print("GeckoDriver and Firefox are working correctly.")
        driver.quit()
    except Exception as e:
        print("An error occurred:", e)


test_geckodriver_direct()

GeckoDriver and Firefox are working correctly.


In [48]:
cities = {
    "Mont Saint Michel": {"dest_id": "900039327", "dest_type": "city"},
    "St Malo": {"dest_id": "-146682", "dest_type": "city"},
    "Bayeux": {"dest_id": "-1410836", "dest_type": "city"},
    "Le Havre": {"dest_id": "-1441598", "dest_type": "city"},
    "Rouen": {"dest_id": "-1462807", "dest_type": "city"},
    "Paris": {"dest_id": "-1456928", "dest_type": "city"},
    "Amiens": {"dest_id": "-1407447", "dest_type": "city"},
    "Lille": {"dest_id": "-1447079", "dest_type": "city"},
    "Strasbourg": {"dest_id": "-1471697", "dest_type": "city"},
    "Chateau du Haut Koenigsbourg": {"dest_id": "204055", "dest_type": "city"},
    "Colmar": {"dest_id": "-1421049", "dest_type": "city"},
    "Eguisheim": {"dest_id": "-1425030", "dest_type": "city"},
    "Besancon": {"dest_id": "-1412198", "dest_type": "city"},
    "Dijon": {"dest_id": "-1423981", "dest_type": "city"},
    "Annecy": {"dest_id": "-1407760", "dest_type": "city"},
    "Grenoble": {"dest_id": "-1430647", "dest_type": "city"},
    "Lyon": {"dest_id": "-1448468", "dest_type": "city"},
    "Gorges du Verdon": {"dest_id": "2746", "dest_type": "region"},
    "Bormes les Mimosas": {"dest_id": "-1413801", "dest_type": "city"},
    "Cassis": {"dest_id": "-1416912", "dest_type": "city"},
    "Marseille": {"dest_id": "-1449947", "dest_type": "city"},
    "Aix en Provence": {"dest_id": "-1406939", "dest_type": "city"},
    "Avignon": {"dest_id": "-1409631", "dest_type": "city"},
    "Uzes": {"dest_id": "-1474231", "dest_type": "city"},
    "Nimes": {"dest_id": "-1455068", "dest_type": "city"},
    "Aigues Mortes": {"dest_id": "-1406800", "dest_type": "city"},
    "Saintes Maries de la mer": {"dest_id": "-1465138", "dest_type": "city"},
    "Collioure": {"dest_id": "-1421032", "dest_type": "city"},
    "Carcassonne": {"dest_id": "-1416701", "dest_type": "city"},
    "Ariege": {"dest_id": "2507", "dest_type": "region"},
    "Toulouse": {"dest_id": "-1473166", "dest_type": "city"},
    "Montauban": {"dest_id": "-1452421", "dest_type": "city"},
    "Biarritz": {"dest_id": "-1412526", "dest_type": "city"},
    "Bayonne": {"dest_id": "-1410844", "dest_type": "city"},
    "La Rochelle": {"dest_id": "-1438604", "dest_type": "city"}
}



base_url = "https://www.booking.com/searchresults.html"


for city, info in cities.items():
    search_url = f"{base_url}?dest_id={info['dest_id']}&dest_type={info['dest_type']}"
    print(f"{city}: {search_url}")


Mont Saint Michel: https://www.booking.com/searchresults.html?dest_id=900039327&dest_type=city
St Malo: https://www.booking.com/searchresults.html?dest_id=-146682&dest_type=city
Bayeux: https://www.booking.com/searchresults.html?dest_id=-1410836&dest_type=city
Le Havre: https://www.booking.com/searchresults.html?dest_id=-1441598&dest_type=city
Rouen: https://www.booking.com/searchresults.html?dest_id=-1462807&dest_type=city
Paris: https://www.booking.com/searchresults.html?dest_id=-1456928&dest_type=city
Amiens: https://www.booking.com/searchresults.html?dest_id=-1407447&dest_type=city
Lille: https://www.booking.com/searchresults.html?dest_id=-1447079&dest_type=city
Strasbourg: https://www.booking.com/searchresults.html?dest_id=-1471697&dest_type=city
Chateau du Haut Koenigsbourg: https://www.booking.com/searchresults.html?dest_id=204055&dest_type=city
Colmar: https://www.booking.com/searchresults.html?dest_id=-1421049&dest_type=city
Eguisheim: https://www.booking.com/searchresults.htm

### Scrapping all hotels web pages

In [ ]:
base_url = "https://www.booking.com/searchresults.html"


def get_hotel_links_with_limit(search_url, max_results=300):
    driver = webdriver.Firefox(service=Service("/snap/bin/geckodriver"))
    driver.get(search_url)
    time.sleep(5)  

    hotel_links = set()  
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while len(hotel_links) < max_results:

        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)  
        
        
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Load more results')]"))
            )
            load_more_button.click()
            time.sleep(3)  
        except:
            pass  
        
        
        all_links = driver.find_elements(By.TAG_NAME, "a")
        for link in all_links:
            href = link.get_attribute("href")
            if href and "hotel" in href:
                hotel_links.add(href.split("?")[0])
                
                
                if len(hotel_links) >= max_results:
                    print(f"Reached the limit of {max_results} hotel URLs for this city.")
                    driver.quit()
                    return list(hotel_links)

        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("Reached the end of the page; no more hotels are loading.")
            break  
        last_height = new_height

    driver.quit()
    
    
    if hotel_links:
        print(f"Found {len(hotel_links)} hotel URLs")
    else:
        print("No hotel links were found on the page.")
    
    return list(hotel_links)


all_hotel_links = {}

for city, info in cities.items():
    search_url = f"{base_url}?dest_id={info['dest_id']}&dest_type={info['dest_type']}"
    print(f"Scraping hotel links for {city}...")
    all_hotel_links[city] = get_hotel_links_with_limit(search_url, max_results=300)


print("All hotel links collected:", all_hotel_links)


Scraping hotel links for Mont Saint Michel...
Reached the end of the page; no more hotels are loading.
Found 76 hotel URLs
Scraping hotel links for St Malo...
Reached the end of the page; no more hotels are loading.
Found 1 hotel URLs
Scraping hotel links for Bayeux...
Reached the end of the page; no more hotels are loading.
Found 208 hotel URLs
Scraping hotel links for Le Havre...
Reached the limit of 300 hotel URLs for this city.
Scraping hotel links for Rouen...
Reached the limit of 300 hotel URLs for this city.
Scraping hotel links for Paris...
Reached the limit of 300 hotel URLs for this city.
Scraping hotel links for Amiens...
Reached the limit of 300 hotel URLs for this city.
Scraping hotel links for Lille...
Reached the limit of 300 hotel URLs for this city.
Scraping hotel links for Strasbourg...
Reached the limit of 300 hotel URLs for this city.
Scraping hotel links for Chateau du Haut Koenigsbourg...
Reached the end of the page; no more hotels are loading.
Found 1 hotel URLs


In [ ]:
special_urls = {
    "St Malo": "https://www.booking.com/searchresults.fr.html?ss=Saint-Malo%2C+Bretagne%2C+France&ssne=Le+Mont-Saint-Michel&ssne_untouched=Le+Mont-Saint-Michel&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARnIAQ_YAQHoAQGIAgGoAgO4AqCd6LkGwAIB0gIkMzJkYzlmMTktOGNiNy00MzY5LWIxNjgtODgyMzExMWZlNTNi2AIF4AIB&sid=6feb6a7600931ed363cc31f799d6ab02&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=-1466824&dest_type=city",
    "Chateau du Haut Koenigsbourg": "https://www.booking.com/searchresults.fr.html?ss=Ch%C3%A2teau+du+Haut-K%C5%93nigsbourg%2C+Saint-Hippolyte%2C+Alsace%2C+France&ssne=Saint-Malo&ssne_untouched=Saint-Malo&label=gen173nr-1BCAEoggI46AdIM1gEaE2IAQGYAQ24ARnIAQ_YAQHoAQGIAgGoAgO4AqCd6LkGwAIB0gIkMzJkYzlmMTktOGNiNy00MzY5LWIxNjgtODgyMzExMWZlNTNi2AIF4AIB&sid=6feb6a7600931ed363cc31f799d6ab02&aid=304142&lang=fr&sb=1&src_elem=sb&src=searchresults&dest_id=204055&dest_type=landmark"
}


def get_hotel_links_with_limit(search_url, max_results=300):
    driver = webdriver.Firefox(service=Service("/snap/bin/geckodriver"))
    driver.get(search_url)
    time.sleep(5)  
    
    hotel_links = set()  
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while len(hotel_links) < max_results:
        
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)  
        
        try:
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Load more results')]"))
            )
            load_more_button.click()
            time.sleep(3)  
        except:
            pass  
        
        
        all_links = driver.find_elements(By.TAG_NAME, "a")
        for link in all_links:
            href = link.get_attribute("href")
            if href and "hotel" in href:
                hotel_links.add(href.split("?")[0])
                
                
                if len(hotel_links) >= max_results:
                    print(f"Reached the limit of {max_results} hotel URLs for this city.")
                    driver.quit()
                    return list(hotel_links)

        
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("Reached the end of the page; no more hotels are loading.")
            break  
        last_height = new_height

    driver.quit()
    
    
    if hotel_links:
        print(f"Found {len(hotel_links)} hotel URLs")
    else:
        print("No hotel links were found on the page.")
    
    return list(hotel_links)


updated_links = {}


for city, url in special_urls.items():
    print(f"Scraping hotel links for {city}...")
    updated_links[city] = get_hotel_links_with_limit(url, max_results=300)

all_hotel_links.update(updated_links)

print("Updated hotel links for St Malo and Chateau du Haut Koenigsbourg:", updated_links)
print("All hotel links with updates:", all_hotel_links)


Scraping hotel links for St Malo...
Reached the end of the page; no more hotels are loading.
Found 76 hotel URLs
Scraping hotel links for Chateau du Haut Koenigsbourg...
Reached the end of the page; no more hotels are loading.
Found 76 hotel URLs
Updated hotel links for St Malo and Chateau du Haut Koenigsbourg: {'St Malo': ['https://www.booking.com/hotel/fr/studio-vue-mer-saint-malo.fr.html', 'https://www.booking.com/hotel/fr/le-skipper-saint-malo.fr.html', 'https://www.booking.com/hotel/fr/la-cote-d-39-emeraude.fr.html', 'https://www.booking.com/hotel/fr/bristol-union.fr.html', 'https://www.booking.com/hotel/fr/les-chiens-du-guet.fr.html', 'https://www.booking.com/hotel/fr/le-valmarin-malouiniere-du-18eme-siecl.fr.html', 'https://www.booking.com/hotel/fr/la-cordee-saint-malo.fr.html', 'https://www.booking.com/hotel/fr/ker-edouard-ty-seb.fr.html', 'https://www.booking.com/hotel/fr/le-croiseur.fr.html', 'https://www.booking.com/hotel/fr/le-port-malo.fr.html', 'https://www.booking.com/ho

### Scrapping all hotels inforamtion

In [ ]:
def scrape_hotel_data(city, url):
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    
    hotel_name = None
    if soup.find("h2", {"class": "hp__hotel-name"}):
        hotel_name = soup.find("h2", {"class": "hp__hotel-name"}).get_text(strip=True)
    elif soup.find("span", {"id": "hp_hotel_name"}):
        hotel_name = soup.find("span", {"id": "hp_hotel_name"}).get_text(strip=True)
    elif soup.find("title"):
        hotel_name = soup.find("title").get_text(strip=True).split(",")[0]

    
    latitude, longitude, rating = None, None, None
    
    for script in soup.find_all("script", type="application/ld+json"):
        try:
            data = json.loads(script.string)
            
            if "geo" in data:
                latitude = data["geo"].get("latitude")
                longitude = data["geo"].get("longitude")
            
            if "aggregateRating" in data:
                rating = data["aggregateRating"].get("ratingValue")
        except (json.JSONDecodeError, TypeError):
            continue
    
    
    if not latitude or not longitude:
        coords_match = re.search(r'"latitude":([0-9.-]+),"longitude":([0-9.-]+)', response.text)
        if coords_match:
            latitude, longitude = coords_match.groups()

    
    if not rating:
        rating_match = re.search(r'"ratingValue":([0-9.]+)', response.text)
        if rating_match:
            rating = rating_match.group(1)
    
    
    description = soup.find("meta", {"name": "description"}).get("content") if soup.find("meta", {"name": "description"}) else None
    
    return {
        "city": city,
        "hotel_name": hotel_name,
        "page_url": url,
        "latitude": latitude,
        "longitude": longitude,
        "rating": rating,
        "description": description
    }


hotel_data = []
for city, urls in all_hotel_links.items():
    for index, url in enumerate(urls, start=1):  
        print(f"Scraping data for city: {city}, hotel #{index}")
        hotel_info = scrape_hotel_data(city, url)
        hotel_data.append(hotel_info)
        time.sleep(2)  


hotel_df = pd.DataFrame(hotel_data)


Scraping data for city: Mont Saint Michel, hotel #1
Scraping data for city: Mont Saint Michel, hotel #2
Scraping data for city: Mont Saint Michel, hotel #3
Scraping data for city: Mont Saint Michel, hotel #4
Scraping data for city: Mont Saint Michel, hotel #5
Scraping data for city: Mont Saint Michel, hotel #6
Scraping data for city: Mont Saint Michel, hotel #7
Scraping data for city: Mont Saint Michel, hotel #8
Scraping data for city: Mont Saint Michel, hotel #9
Scraping data for city: Mont Saint Michel, hotel #10
Scraping data for city: Mont Saint Michel, hotel #11
Scraping data for city: Mont Saint Michel, hotel #12
Scraping data for city: Mont Saint Michel, hotel #13
Scraping data for city: Mont Saint Michel, hotel #14
Scraping data for city: Mont Saint Michel, hotel #15
Scraping data for city: Mont Saint Michel, hotel #16
Scraping data for city: Mont Saint Michel, hotel #17
Scraping data for city: Mont Saint Michel, hotel #18
Scraping data for city: Mont Saint Michel, hotel #19
Sc

### Add cities lattitude and longitude to hotel_data

In [6]:
hotel_df = hotel_df.merge(city_df[['city_name', 'city_latitude', 'city_longitude']], 
                          left_on='city', right_on='city_name', how='left')

hotel_df = hotel_df.drop(columns=['city_name'])

In [8]:
hotel_df

,city,hotel_name,page_url,latitude,longitude,rating,description,city_latitude,city_longitude
0,Mont Saint Michel,Maison neuve à deux pas du Mont,https://www.booking.com/hotel/fr/maison-neuve-...,48.597124,-1.503780,9.0,Maison neuve à deux pas du Mont offers accommo...,48.6359541,-1.511459954959514
1,Mont Saint Michel,Gîte Grand'elle,https://www.booking.com/hotel/fr/gite-grand-el...,48.596165,-1.504250,10.0,"Gîte Grand'elle is located in Beauvoir, just 2...",48.6359541,-1.511459954959514
2,Mont Saint Michel,O rivage du Mont 5 pers,https://www.booking.com/hotel/fr/o-rivage-du-m...,48.619310,-1.459190,8.2,"Featuring a garden, O rivage du Mont 5 pers fe...",48.6359541,-1.511459954959514
3,Mont Saint Michel,Vent des Grèves,https://www.booking.com/hotel/fr/vent-des-grev...,48.615403,-1.491440,9.2,Vent des Grèves is a recently renovated bed an...,48.6359541,-1.511459954959514
4,Mont Saint Michel,Le Mouton Blanc,https://www.booking.com/hotel/fr/le-mouton-bla...,48.636023,-1.509896,7.2,"At the foot of the abbey, the Mouton Blanc Hot...",48.6359541,-1.511459954959514
...,...,...,...,...,...,...,...,...,...
8647,La Rochelle,Appartement 6 places à 200m de l'océan,https://www.booking.com/hotel/fr/appartement-6...,46.139170,-1.158884,8.8,Just a 9-minute walk from Plage Du Roux and 0....,46.159732,-1.1515951
8648,La Rochelle,Terre en Vue,https://www.booking.com/hotel/fr/terre-en-vue....,46.141926,-1.134727,9.2,Just 1.1 miles from Exhibition Park of La Roch...,46.159732,-1.1515951
8649,La Rochelle,Studio neuf proche Océan,https://www.booking.com/hotel/fr/studio-neuf-p...,46.139539,-1.139870,8.2,Studio neuf proche Océan offers accommodations...,46.159732,-1.1515951
8650,La Rochelle,L'Écrin De L'Hyper Centre,https://www.booking.com/hotel/fr/ecrin-de-l-hy...,46.164309,-1.148915,8.7,L'Écrin De L'Hyper Centre in La Rochelle provi...,46.159732,-1.1515951


### Save datas

In [9]:
hotel_df.to_csv('hotel_information.csv', index=False)

In [10]:
csv_file_path = 'hotel_information.csv'
hotel_df.to_csv(csv_file_path, index=False)

s3 = boto3.client('s3')

bucket_name = 'jedhajules'
s3_key = 'hotel_information.csv'

s3.upload_file(csv_file_path, bucket_name, s3_key)

print(f"File successfully uploaded to s3://{bucket_name}/{s3_key}")

File successfully uploaded to s3://jedhajules/hotel_information.csv


### Loading data to RDS database

checking connection

In [ ]:
rds_host = 'database-hotels.cf2yiegy6kh1.eu-west-3.rds.amazonaws.com'  
rds_user = 'postgres'
rds_password = ''
rds_dbname = 'postgres'

In [ ]:
try:
    
    conn = psycopg2.connect(
        host=rds_host,
        database=rds_dbname,
        user=rds_user,
        password=rds_password
    )
    print("Connected to PostgreSQL database successfully.")
    
    
    conn.close()

except Exception as e:
    print(f"Failed to connect to PostgreSQL database: {e}")


Connected to PostgreSQL database successfully.


loading data

In [ ]:
bucket_name = 'jedhajules'
s3_file_key = 'hotel_information.csv'

s3 = boto3.client('s3')
csv_obj = s3.get_object(Bucket=bucket_name, Key=s3_file_key)
csv_data = csv_obj['Body'].read().decode('utf-8')


hotel_df = pd.read_csv(io.StringIO(csv_data))


conn = psycopg2.connect(
    host=rds_host,
    database=rds_dbname,
    user=rds_user,
    password=rds_password
)
cur = conn.cursor()


create_table_query = """
CREATE TABLE IF NOT EXISTS hotel_information (
    city VARCHAR(100),
    hotel_name VARCHAR(255),
    page_url TEXT,
    latitude FLOAT,
    longitude FLOAT,
    rating FLOAT,
    description TEXT,
    city_latitude FLOAT,
    city_longitude FLOAT

);
"""
cur.execute(create_table_query)
conn.commit()


for index, row in hotel_df.iterrows():
    insert_query = """
    INSERT INTO hotel_information (city, hotel_name, page_url, latitude, longitude, rating, description, city_latitude, city_longitude)
    VALUES (%s, %s, %s, %s, %s, %s, %s,%s, %s);
    """
    cur.execute(insert_query, (
        row['city'],
        row['hotel_name'],
        row['page_url'],
        row['latitude'],
        row['longitude'],
        row['rating'],
        row['description'],
        row['city_latitude'],
        row['city_longitude']
    ))
    conn.commit()

print("Data loaded successfully to RDS.")


cur.close()
conn.close()


Data loaded successfully to RDS.


checking if everuthing worked

In [ ]:

try:
    conn = psycopg2.connect(
        host=rds_host,
        database=rds_dbname,
        user=rds_user,
        password=rds_password
    )
    cur = conn.cursor()

    
    query = "SELECT * FROM hotel_information LIMIT 10;"  
    cur.execute(query)

    
    rows = cur.fetchall()
    for row in rows:
        print(row)

    
    cur.close()
    conn.close()

except Exception as e:
    print(f"Failed to connect or query PostgreSQL database: {e}")


('Mont Saint Michel', 'Maison neuve à deux pas du Mont', 'https://www.booking.com/hotel/fr/maison-neuve-a-deux-pas-du-mont-4-6-personnes.html', 48.5971241, -1.5037796, 9.0, 'Maison neuve à deux pas du Mont offers accommodations in Beauvoir, 28 miles from Port of Houle and 30 miles from Granville Train Station.', 48.6359541, -1.511459954959514)
('Mont Saint Michel', "Gîte Grand'elle", 'https://www.booking.com/hotel/fr/gite-grand-elle.html', 48.5961652577708, -1.50425000886945, 10.0, "Gîte Grand'elle is located in Beauvoir, just 28 miles from Port of Houle and 29 miles from Granville Train Station.", 48.6359541, -1.511459954959514)
('Mont Saint Michel', 'O rivage du Mont 5 pers', 'https://www.booking.com/hotel/fr/o-rivage-du-mont.html', 48.619310175218, -1.45919002901, 8.2, 'Featuring a garden, O rivage du Mont 5 pers features accommodations in Huisnes-sur-Mer.', 48.6359541, -1.511459954959514)
('Mont Saint Michel', 'Vent des Grèves', 'https://www.booking.com/hotel/fr/vent-des-greves-pon